In [80]:
from flask import Flask
from flask import render_template,request
from datetime import datetime
import mysql.connector
import pandas as pd
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler

In [81]:
def item_call(item_id,db):
    query=f'call item_call({item_id})'
    results=db.execute(query,multi=True)
    data=[]
    try:
        for i in results:
            data=i.fetchall()
    except:
        pass
    print(data)
    data=pd.DataFrame(data,columns=['order_date','order_hour','quantity'])
    data['order_day']=list(map((lambda x:x.weekday()),data['order_date'].to_list()))
    return data

In [82]:
def predict(data,day,time):
    scaler=StandardScaler()
    data[['order_day','order_hour']]=scaler.fit_transform(data[['order_day','order_hour']])
    x_train,y_train=data[['order_day','order_hour']],data['quantity']
    linear=linear_model.LinearRegression()
    reg=linear.fit(x_train,y_train)
    result=reg.predict([[day,time]])
    result=result.round()
    return result

In [83]:
mydb=mysql.connector.connect(host="127.0.0.1",user="root", passwd="Govinda457", database='leftovers1')
my=mydb.cursor(buffered=True)

In [84]:
my.callproc('total_orders')

()

In [85]:
app =Flask(__name__,template_folder='templates') 
@app.route("/",methods = ['POST','GET'])  
def result():
  if request.method=='POST':
    date=request.form['mydate']
    time=request.form['mytime']
    item_id=request.form['item_id']
    day= datetime.strptime(date,'%Y-%m-%d').weekday()
    data=item_call(int(item_id),my)
    result=predict(data,int(day),int(time))
    return render_template('input.html', msg1 =abs(result[0]))  
  return render_template('input.html', message = "")

if __name__ =='__main__':  
    app.run(debug = True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Aug/2024 00:03:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2024 00:03:02] "GET /static/style2.css HTTP/1.1" 304 -
127.0.0.1 - - [13/Aug/2024 00:03:02] "GET /static/minimal-geometric-stripe-shape-background_1409-1014.webp HTTP/1.1" 304 -
C:\Users\indra\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
127.0.0.1 - - [13/Aug/2024 00:03:15] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2024 00:03:15] "GET /static/style2.css HTTP/1.1" 304 -


[(datetime.date(2023, 1, 1), 10, 0), (datetime.date(2023, 1, 1), 11, 0), (datetime.date(2023, 1, 1), 12, 2), (datetime.date(2023, 1, 1), 13, 0), (datetime.date(2023, 1, 1), 14, 0), (datetime.date(2023, 1, 1), 15, 0), (datetime.date(2023, 1, 1), 16, 0), (datetime.date(2023, 1, 1), 17, 0), (datetime.date(2023, 1, 1), 18, 1), (datetime.date(2023, 1, 1), 19, 0), (datetime.date(2023, 1, 1), 20, 1), (datetime.date(2023, 1, 1), 21, 1), (datetime.date(2023, 1, 1), 22, 0), (datetime.date(2023, 1, 1), 23, 0), (datetime.date(2023, 1, 2), 10, 0), (datetime.date(2023, 1, 2), 11, 0), (datetime.date(2023, 1, 2), 12, 1), (datetime.date(2023, 1, 2), 13, 0), (datetime.date(2023, 1, 2), 14, 2), (datetime.date(2023, 1, 2), 15, 1), (datetime.date(2023, 1, 2), 16, 0), (datetime.date(2023, 1, 2), 17, 0), (datetime.date(2023, 1, 2), 18, 2), (datetime.date(2023, 1, 2), 19, 0), (datetime.date(2023, 1, 2), 20, 0), (datetime.date(2023, 1, 2), 21, 1), (datetime.date(2023, 1, 2), 22, 1), (datetime.date(2023, 1, 2),

C:\Users\indra\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
127.0.0.1 - - [13/Aug/2024 00:03:33] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2024 00:03:33] "GET /static/style2.css HTTP/1.1" 304 -


[(datetime.date(2023, 1, 1), 10, 0), (datetime.date(2023, 1, 1), 11, 0), (datetime.date(2023, 1, 1), 12, 1), (datetime.date(2023, 1, 1), 13, 1), (datetime.date(2023, 1, 1), 14, 3), (datetime.date(2023, 1, 1), 15, 1), (datetime.date(2023, 1, 1), 16, 0), (datetime.date(2023, 1, 1), 17, 1), (datetime.date(2023, 1, 1), 18, 0), (datetime.date(2023, 1, 1), 19, 0), (datetime.date(2023, 1, 1), 20, 0), (datetime.date(2023, 1, 1), 21, 1), (datetime.date(2023, 1, 1), 22, 0), (datetime.date(2023, 1, 1), 23, 0), (datetime.date(2023, 1, 2), 10, 0), (datetime.date(2023, 1, 2), 11, 0), (datetime.date(2023, 1, 2), 12, 2), (datetime.date(2023, 1, 2), 13, 0), (datetime.date(2023, 1, 2), 14, 1), (datetime.date(2023, 1, 2), 15, 0), (datetime.date(2023, 1, 2), 16, 1), (datetime.date(2023, 1, 2), 17, 0), (datetime.date(2023, 1, 2), 18, 1), (datetime.date(2023, 1, 2), 19, 0), (datetime.date(2023, 1, 2), 20, 1), (datetime.date(2023, 1, 2), 21, 1), (datetime.date(2023, 1, 2), 22, 0), (datetime.date(2023, 1, 2),

...

[(datetime.date(2023, 1, 1), 10, 0), (datetime.date(2023, 1, 1), 11, 0), (datetime.date(2023, 1, 1), 12, 1), (datetime.date(2023, 1, 1), 13, 1), (datetime.date(2023, 1, 1), 14, 3), (datetime.date(2023, 1, 1), 15, 1), (datetime.date(2023, 1, 1), 16, 0), (datetime.date(2023, 1, 1), 17, 1), (datetime.date(2023, 1, 1), 18, 0), (datetime.date(2023, 1, 1), 19, 0), (datetime.date(2023, 1, 1), 20, 0), (datetime.date(2023, 1, 1), 21, 1), (datetime.date(2023, 1, 1), 22, 0), (datetime.date(2023, 1, 1), 23, 0), (datetime.date(2023, 1, 2), 10, 0), (datetime.date(2023, 1, 2), 11, 0), (datetime.date(2023, 1, 2), 12, 2), (datetime.date(2023, 1, 2), 13, 0), (datetime.date(2023, 1, 2), 14, 1), (datetime.date(2023, 1, 2), 15, 0), (datetime.date(2023, 1, 2), 16, 1), (datetime.date(2023, 1, 2), 17, 0), (datetime.date(2023, 1, 2), 18, 1), (datetime.date(2023, 1, 2), 19, 0), (datetime.date(2023, 1, 2), 20, 1), (datetime.date(2023, 1, 2), 21, 1), (datetime.date(2023, 1, 2), 22, 0), (datetime.date(2023, 1, 2),

C:\Users\indra\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
127.0.0.1 - - [13/Aug/2024 00:04:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2024 00:04:03] "GET /static/style2.css HTTP/1.1" 304 -


[(datetime.date(2023, 1, 1), 10, 0), (datetime.date(2023, 1, 1), 11, 0), (datetime.date(2023, 1, 1), 12, 2), (datetime.date(2023, 1, 1), 13, 0), (datetime.date(2023, 1, 1), 14, 0), (datetime.date(2023, 1, 1), 15, 0), (datetime.date(2023, 1, 1), 16, 0), (datetime.date(2023, 1, 1), 17, 0), (datetime.date(2023, 1, 1), 18, 1), (datetime.date(2023, 1, 1), 19, 0), (datetime.date(2023, 1, 1), 20, 1), (datetime.date(2023, 1, 1), 21, 1), (datetime.date(2023, 1, 1), 22, 0), (datetime.date(2023, 1, 1), 23, 0), (datetime.date(2023, 1, 2), 10, 0), (datetime.date(2023, 1, 2), 11, 0), (datetime.date(2023, 1, 2), 12, 1), (datetime.date(2023, 1, 2), 13, 0), (datetime.date(2023, 1, 2), 14, 2), (datetime.date(2023, 1, 2), 15, 1), (datetime.date(2023, 1, 2), 16, 0), (datetime.date(2023, 1, 2), 17, 0), (datetime.date(2023, 1, 2), 18, 2), (datetime.date(2023, 1, 2), 19, 0), (datetime.date(2023, 1, 2), 20, 0), (datetime.date(2023, 1, 2), 21, 1), (datetime.date(2023, 1, 2), 22, 1), (datetime.date(2023, 1, 2),

C:\Users\indra\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
127.0.0.1 - - [13/Aug/2024 00:04:22] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2024 00:04:22] "GET /static/style2.css HTTP/1.1" 304 -


[(datetime.date(2023, 1, 1), 10, 0), (datetime.date(2023, 1, 1), 11, 1), (datetime.date(2023, 1, 1), 12, 5), (datetime.date(2023, 1, 1), 13, 2), (datetime.date(2023, 1, 1), 14, 0), (datetime.date(2023, 1, 1), 15, 1), (datetime.date(2023, 1, 1), 16, 1), (datetime.date(2023, 1, 1), 17, 2), (datetime.date(2023, 1, 1), 18, 0), (datetime.date(2023, 1, 1), 19, 0), (datetime.date(2023, 1, 1), 20, 1), (datetime.date(2023, 1, 1), 21, 0), (datetime.date(2023, 1, 1), 22, 2), (datetime.date(2023, 1, 1), 23, 0), (datetime.date(2023, 1, 2), 10, 0), (datetime.date(2023, 1, 2), 11, 0), (datetime.date(2023, 1, 2), 12, 1), (datetime.date(2023, 1, 2), 13, 1), (datetime.date(2023, 1, 2), 14, 0), (datetime.date(2023, 1, 2), 15, 2), (datetime.date(2023, 1, 2), 16, 0), (datetime.date(2023, 1, 2), 17, 1), (datetime.date(2023, 1, 2), 18, 2), (datetime.date(2023, 1, 2), 19, 1), (datetime.date(2023, 1, 2), 20, 0), (datetime.date(2023, 1, 2), 21, 0), (datetime.date(2023, 1, 2), 22, 0), (datetime.date(2023, 1, 2),